<a href="https://colab.research.google.com/github/JF11579/Plotly_RE/blob/main/Condo_Comparison_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [3]:
!pip install geopandas
!pip install geopy

In [13]:
import pandas as pd
from google.colab import drive
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import plotly.express as px

# Options

In [4]:
# Set pandas display options to show more rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

# Data

In [5]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Change to the directory containing your CSV
%cd /content/drive/MyDrive/DATA/Real_Estate_Data
#

Mounted at /content/drive
/content/drive/MyDrive/DATA/Real_Estate_Data


In [6]:
condo = pd.read_csv('Coram_North_Isle.csv')

## Data cleaning

In [7]:
condo.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)

In [8]:
condo['Bedrooms'] = condo['Bedrooms'].astype(int)
condo['Unit_on_Floor_#'] = condo['Unit_on_Floor_#'].astype(int)

In [9]:
condo = condo[['Address', 'Town','Zip','State','Style','Street_Name',
                     'Development','ML#','Sold_Price','Baths_Full', 'Baths_Half',
                   'Room_Level','Bath','Bedrooms','Utilities_Included', 'Common_Charges',
                     'Appearance','REO','Appearance','Approx_Int_Square_Footage','School_District_Name',
               'Taxes' ,'Unit_on_Floor_#']]

In [10]:
condo = condo.drop_duplicates(subset=['ML#'], keep='first')

In [11]:
# convert SoldPRice to a number
condo['Sold_Price'] = pd.to_numeric(condo['Sold_Price'], errors='coerce')

# Sales by Devlopments Plotted(color by bedroom count)

In [14]:
fig = px.scatter(data_frame=condo, x='Development', y='Sold_Price', color='Bedrooms',
                 title='Condo Sold Price vs. Development',
                 labels={'Development': 'Development', 'sold Price': 'Sold Price', 'Town': 'Town'},
                 hover_data=['Development'])

fig.update_traces(hovertemplate='Development: %{x}<br>Sold Price: %{y}<br>Bedrooms: %{marker.color}')

fig.show()

# Plot Facet by Bedroom

In [15]:
fig = px.scatter(data_frame=condo, x='Development', y='Sold_Price', color='Town',
                 title='Condo Sold Price vs. Development',
                 labels={'Development': 'Development', 'sold Price': 'Sold Price', 'Bedrooms': 'Number of Bedrooms'},
                 facet_col='Bedrooms',  # Create subplots for each value of 'Bedrooms'
                 height=400)  # Set the height of each subplot

# Show the plot
fig.show()

# Plot Square footage v Selling Price

In [19]:
# Create a scatter plot using Plotly Express
fig = px.scatter(condo_area , x='Bedrooms', y='Approx_Int_Square_Footage',color='Sold_Price' ,title='Sold Price and Sq Footage')
#
# Set y-axis type to linear to ensure numeric order
fig.update_yaxes(type='linear')

# Update the x-axis to show only integer ticks
fig.update_xaxes(
    tickmode='linear',   # Use a linear tick mode
    tick0=1,             # Start the ticks from 1 (an integer)
    dtick=1              # Set the tick interval to 1 (integer increment)
)
# Show the plot
fig.show()

# School District and Selling Price

In [20]:
School_dist_data = condo

unique_school_district = School_dist_data['School_District_Name'].unique()
unique_school_district

fig = px.scatter(School_dist_data, x='School_District_Name', y='Sold_Price', title='Scatter Plot of Sold Price vs. School District')

# Show the plot
fig.show()

# Plot: Unit Level and Selling Price

In [22]:
condo.head(3)

,Address,Town,Zip,State,Style,Street_Name,Development,ML#,Sold_Price,Baths_Full,Baths_Half,Room_Level,Bath,Bedrooms,Utilities_Included,Common_Charges,Appearance,REO,Appearance,Approx_Int_Square_Footage,School_District_Name,Taxes,Unit_on_Floor_#
0,6 Redwood Court,Coram,11727,NY,Townhouse,Redwood,North Isle,3367901,145000.0,1,0,2,1.0,1,"Garbage Removal,Hot Water,Pool Care,Sewer,Water",662.0,Mint,N,Mint,NaN,Longwood,0.0,2
1,182 Pointe Circle No,Coram,11727,NY,Other,Pointe Circle No,North Isle Village,3368100,183000.0,1,0,"2,2,2,2,2",1.0,2,"Air Conditioning,Garbage Removal,Gas,Ground Ca...",NaN,Diamond,N,Diamond,NaN,Longwood,0.0,2
2,12 Fargo Court,Coram,11727,NY,Garden Apartment,Fargo,North Isle,3370227,130000.0,1,0,2,1.0,1,"Cable,Sewer,Water",641.0,very good,N,very good,NaN,Longwood,0.0,1


In [23]:
 FloorLevel = condo

 # Unit on Floor #
fig = px.scatter(FloorLevel, x='Unit_on_Floor_#', y='Sold_Price', title='Sold Price vs. Floor Level')

# Update the x-axis to show only integer ticks
fig.update_xaxes(
    tickmode='linear',   # Use a linear tick mode
    tick0=1,             # Start the ticks from 1 (an integer)
    dtick=1              # Set the tick interval to 1 (integer increment)
)

# Show the plot
fig.show()

# Appearance

In [24]:
appearance_data = condo

def map_existing_values(value):
    if pd.isna(value):  # Handle NaN values explicitly
        return 'unknown'
    if value == "Appearance":
        return 0
    elif value in ["AS IS"]:
        return 'As Is'
    elif value in ["Fair", "good", "Good", "Very Good", "Very good"]:
        return 'Fair'
    elif value in ["Excellent", "EXCELLENT"]:
        return 'Excellent'
    elif value == "Move in":
        return 'Move In'
    elif value in ["diamond", "Diamond", "Mint", "mint", "Mint+", "Mint++", "Mint+++", "Mint+++"]:
        return 'Diamond'
    elif value == "unknown":
        return 'unknown'
    elif value == "unknown":
        return 'unknown'
    else:
        return -1  # Return -1 for other cases or return None if you prefer


df = pd.DataFrame({'condition': ['Appearance', 'AS IS', 'Good', 'Excellent', 'Move in', 'Mint']})

condition_new = df['condition'].map(map_existing_values)

appearance_data = appearance_data.assign(appearance_data_cat=condition_new)

fig = px.scatter(appearance_data, x='appearance_data_cat', y='Sold_Price',facet_col='Bedrooms', title='Scatter Plot of Sold Price vs. Condition')

# Show the plot
fig.show()

# Common_Cahrge v Selling Price

In [25]:
Taxes = condo

fig = px.scatter(Taxes, x='Common_Charges', y='Sold_Price', title='Scatter Plot of Sold Price vs. Bedrooms')

# Update the x-axis to show only integer ticks
fig.update_xaxes(
    tickmode='linear',   # Use a linear tick mode
    tick0=1,             # Start the ticks from 1 (an integer)
    dtick=1              # Set the tick interval to 1 (integer increment)
)

# Show the plot
fig.show()

# Common Charge v Sq Feet

In [27]:
CommonCharge = condo

fig = px.scatter(CommonCharge, x='Common_Charges', y='Approx_Int_Square_Footage', color='Bedrooms',
                 color_discrete_map={'1': 'blue', '2': 'green', '3': 'red'})  # Add more colors if needed

# Update layout to set background color to black
fig.update_layout(
    plot_bgcolor='grey',   # Background color of the plot area
    paper_bgcolor='grey'   # Background color of the entire figure
)

#
# Hide the color bar legend
fig.update_layout(showlegend=False)
#

fig.show()